In [ ]:
import numpy as np
import pandas as pd

In [ ]:
'''
def funcion_prioridad_algoles(prioridad='almacen'):
    if prioridad == 'almacen':
        algo4()
        algo5()
    elif prioridad == 'produccion':
        algo5()
        algo4()
    else:
        print('No existe esa prioridad. Favor hablar con Ale')
    return
'''


#GENERAR WARNING SI EXISTE STOCK EN ALMACENES DE ANTIGUEDAD FUERA DE LOS RANGOS PROPUESTOS.

In [ ]:
#temporada: verano, 1 es verano, 0 es invierno.
inv_verano = pd.read_excel('../02_data/verano.xlsx', sheet_name='veranos', skiprows=1)
inv_verano = pd.melt(inv_verano, id_vars=['zona'], value_vars=inv_verano.columns[1:] , var_name='mes_no', value_name='temporada')


import datetime
import dateutil.relativedelta as relativedelta

mes_inicio = 9
año_inicio = 2023

date_inicio = datetime.datetime(year=año_inicio, month=mes_inicio, day=10)
primer_periodo = str(date_inicio.year) + '-' + str(date_inicio.month).zfill(2)

periodos = []
horizonte_planificacion = 12
for mes in range(horizonte_planificacion + 1):
    aux_per = date_inicio + relativedelta.relativedelta(months=mes)
    periodos.append(str(aux_per.year) + '-' + str(aux_per.month).zfill(2))

periodos = pd.DataFrame(periodos, columns=['mes_real'])
periodos['año'] = periodos['mes_real'].str[:4]
periodos['mes_no'] = periodos['mes_real'].str[5:7].astype(int)
periodos['mes_planificado'] = periodos.index + 1
periodos = pd.merge(periodos, inv_verano, how='left', on='mes_no')
periodos.head()

In [ ]:
rango_mes_tipo = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='rango_mes_tipo')
dict_tiporango = rango_mes_tipo.set_index(['mes_planificado', 'nombre_planta', 'producto'])['id_tipo_rango'].to_dict()

rangos_demanda = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='parametro_antiguedad')

rangos_demanda_ext = pd.DataFrame(columns=['id_tipo_rango', 'rango', 'antiguedad'])
for ix in rangos_demanda.index:
    for ant in range(rangos_demanda.at[ix, 'inicio_rango'], rangos_demanda.at[ix, 'fin_rango'] + 1):
        aux = [[rangos_demanda.at[ix, 'id_tipo_rango'], rangos_demanda.at[ix, 'rango'], ant ]]
        aux = pd.DataFrame(aux, columns=['id_tipo_rango', 'rango', 'antiguedad'])
        rangos_demanda_ext = pd.concat([rangos_demanda_ext, aux], ignore_index=True)

rangos_demanda_ext['antiguedad'] = rangos_demanda_ext['antiguedad'].astype(np.int64).astype(str)

In [ ]:
#Lectura de parametros planta
parametros_planta = pd.read_excel('../02_data/plantas.xlsx', sheet_name='parametros_planta')
parametros_planta['stock_ideal_planta'] = parametros_planta['ideal_dias_autonomia'] * parametros_planta['consumo_diario'] 

#Lectura consumo plantas
consumo_plantas = pd.read_excel('../02_data/plantas.xlsx', sheet_name='consumo_plantas')
consumo_plantas = pd.melt(consumo_plantas, consumo_plantas.columns[:2], consumo_plantas.columns[2:], var_name='mes_planificado', value_name='volumen_consumo')

## Matriz de demandas.

#Consumo_planta_tipo + parametros_planta_tipo
#Calculo demanda
df = pd.merge(consumo_plantas, parametros_planta, on=['nombre_planta', 'producto'])
df.loc[df['mes_planificado'] != 1, 'stock_inicial'] = 0
df['demanda'] = df['volumen_consumo'] + df['stock_ideal_planta'] - df['stock_inicial']

#Agregar los tipos de rango por mes
df = pd.merge(df, rango_mes_tipo, how='left', on=['nombre_planta', 'producto', 'mes_planificado'])
df = pd.merge(df, rangos_demanda, how='left', on='id_tipo_rango')
df['demanda_rango'] = df['demanda'] * df['porcentaje']
df = df[['nombre_planta', 'producto', 'mes_planificado', 'id_tipo_rango', 'rango', 'inicio_rango', 'fin_rango', 'demanda_rango']].dropna()
df['id_tipo_rango'] = df['id_tipo_rango'].astype(int)
df['inicio_rango'] = df['inicio_rango'].astype(int)
df['fin_rango'] = df['fin_rango'].astype(int)

demanda_plantas = df.copy()
demanda_plantas.to_csv('../03_pydata/demanda_plantas.csv', index=False, sep=';', decimal=',')

In [ ]:
#Carga Ingresos: Compras y produccion
base_ingresos = pd.read_excel('../02_data/ingresos.xlsx', sheet_name='ingresos')
display(base_ingresos.head())

#Carga Ingreso: Volumen mensual
vol_ingresos = pd.read_excel('../02_data/ingresos_volumen.xlsx', sheet_name='ingresovolumen', skiprows=1)
vol_ingresos = pd.melt(vol_ingresos, vol_ingresos.columns[:1], vol_ingresos.columns[1:], var_name='mes_planificado', value_name='volumen')
vol_ingresos = pd.merge(base_ingresos, vol_ingresos, on='id_ingreso', how='left')
vol_ingresos.to_csv('../03_pydata/ingresos_volumen.csv', sep=';', decimal=',', index=False)

In [ ]:
### Carga de Almacenes ###
base_almacenes = pd.read_excel('../02_data/almacenes.xlsx', sheet_name='almacenes')
no_almacenes = len(base_almacenes)

alm_mes_temp = base_almacenes[['id_almacen', 'zona', 'carpeta']].copy().drop_duplicates(ignore_index=True)
alm_mes_temp = pd.merge(alm_mes_temp, periodos, how='left', on='zona')
alm_mes_temp = alm_mes_temp[['id_almacen', 'carpeta', 'mes_real', 'zona', 'mes_planificado', 'temporada']].copy().drop_duplicates(ignore_index=True)
alm_mes_temp['bloqueo'] = False
alm_mes_temp.loc[(alm_mes_temp['carpeta'] == 'tierra') & (alm_mes_temp['temporada'] == 0), 'bloqueo'] = True
dict_temp_almacen = alm_mes_temp.set_index(['id_almacen', 'mes_planificado'])['bloqueo'].to_dict()

#Stock almacen EN MES 0
stock_almacen = pd.read_excel('../02_data/almacenes_stock.xlsx', sheet_name='cierrestock', skiprows=1)
stock_almacen = pd.melt(stock_almacen, stock_almacen.columns[:1], stock_almacen.columns[1:], var_name='antiguedad', value_name='stock_inicial').fillna(0)

stock_almacen = pd.merge(base_almacenes, stock_almacen, how='left', on='id_almacen')
stock_almacen['antiguedad'] = stock_almacen['antiguedad'].astype(np.int64).astype(str)
stock_almacen.insert(loc=len(stock_almacen.columns)-1, column='mes_planificado', value=1)
stock_almacen['ventas'] = 0
stock_almacen['salida_acanche'] = 0
stock_almacen['ingreso_libre'] = 0
stock_almacen['ingreso_almacen'] = 0
stock_almacen['stock_final'] = 0
stock_almacen

In [ ]:
periodos
rango_mes_tipo
dict_tiporango
rangos_demanda
rangos_demanda_ext
dict_temp_almacen

stock_almacen
vol_ingresos
demanda_plantas

In [ ]:
tabla_demanda = pd.DataFrame()
tabla_ventas_ingresos = pd.DataFrame()
tabla_ventas_almacenes = pd.DataFrame()

for mes in [1]:
    for planta in df['nombre_planta'].unique():
        # for producto in df[df['nombre_planta'] == planta]['producto'].unique():
        for producto in ['EUGL CC']:
            print(planta, producto, mes)

            tipo_rango = dict_tiporango[(mes, planta, producto)]
            rde = rangos_demanda_ext.copy()
            rde = rde.loc[rde['id_tipo_rango'] == tipo_rango].drop(columns=['id_tipo_rango'])

            #Demanda del mes
            dpla = demanda_plantas.copy()
            dpla = dpla[(dpla['nombre_planta'] == planta) & (dpla['producto'] == producto) & (dpla['mes_planificado'] == mes)]

            #Almacenes disponibles
            alm_disponible = alm_mes_temp.copy()
            alm_disponible = alm_disponible[(alm_disponible['mes_planificado'] == mes) & (alm_disponible['bloqueo'] == False)]
            alm_disponible = list(alm_disponible['id_almacen'].unique())

            #Almacenes con rango 
            alma = stock_almacen.copy()
            alma = alma[(alma['id_almacen'].isin(alm_disponible))
                        & (alma['nombre_planta'] == planta) 
                        & (alma['producto'] == producto) 
                        & (alma['mes_planificado'] == mes)]
            alma = pd.merge(alma, rde, how='left', on=['antiguedad'])

            if len(alma[alma['rango'].isnull()]) > 0:
                print('ERROR: las siguientes antiguedades no tienen un rango en el "tipo_rango" ')
                print(alma[alma['rango'].isnull()]['antiguedad'].unique())

            #ingdir: ingresos directos
            ingdir = vol_ingresos.copy()
            ingdir = ingdir.loc[(ingdir['nombre_planta'] == planta) 
                            & (ingdir['producto'] == producto) 
                            & (ingdir['mes_planificado'] == mes)
                            & (ingdir['tipo_ingreso'] == 'directo_planta')]
            
            #inglib: ingresos libres
            inglib = vol_ingresos.copy()
            inglib = inglib.loc[(ingdir['nombre_planta'] == planta) 
                & (inglib['producto'] == producto) 
                & (inglib['mes_planificado'] == mes)
                & (inglib['tipo_ingreso'] == 'libre')]
            


            

            print('Algo 1: Modificacion de demandas. Desde lo mas antiguo a lo mas nuevo')
            #Si la planta no tiene demanda en el mes actual, se continua el ciclo.
            if len(dpla) > 0:
                dpla = dpla.sort_values(by='inicio_rango', ascending=False)
                
                alma_aux = alma.groupby(['nombre_planta', 'producto', 'rango'], as_index=False).agg(volumen=('stock_inicial', 'sum'))
                alma_aux = alma_aux.sort_values(by='rango', ascending=False).reset_index(drop=True)

                dpla = pd.merge(dpla, alma_aux, how='left', on=['nombre_planta', 'producto', 'rango'])
                dpla['volumen'] = dpla['volumen'].fillna(0)

                #Algoritmo Ale
                for ix in dpla.index[:-1]:
                    if dpla.at[ix, 'volumen'] < dpla.at[ix, 'demanda_rango']:
                        dpla.at[ix + 1, 'demanda_rango'] +=  dpla.at[ix, 'demanda_rango'] - dpla.at[ix, 'volumen']
                        dpla.at[ix, 'demanda_rango'] = dpla.at[ix, 'volumen']
                dpla = dpla.drop(columns=['volumen']).sort_values(by=['mes_planificado', 'inicio_rango'])





            print('Algo 2: Modificacion de demandas. El ingreso directo debe estar complemente incluido en la demanda')
            if len(dpla) > 0:
                dpla = dpla.sort_values(by='inicio_rango', ascending=True).reset_index(drop=True)
                vol_demanda = dpla['demanda_rango'].sum()
                
                #Se filtran los ingresos de planta directo.
                aux_ingdir = ingdir.groupby(['nombre_planta', 'producto', 'mes_planificado'], as_index=False).agg({'volumen':'sum'})

                #Indicadores
                vol_fresco = aux_ingdir.at[0, 'volumen']
                vol_stock = vol_fresco - vol_demanda
                if vol_stock < 0:
                    vol_stock = 0.0

                print(f'vol_demanda: {vol_demanda}, vol_ingreso_directo: {vol_fresco}, vol_stock: {vol_stock}')

                #Algoritmo Ale
                if vol_fresco > dpla.at[0, 'demanda_rango']:
                    dpla.at[1, 'demanda_rango'] = dpla.at[1, 'demanda_rango'] + dpla.at[0, 'demanda_rango'] - vol_fresco
                    dpla.at[0, 'demanda_rango'] = vol_fresco

                for ix in dpla.index[1:-1]:
                    if dpla.at[ix, 'demanda_rango'] < 0:
                        dpla.at[ix+1, 'demanda_rango'] = dpla.at[ix+1, 'demanda_rango'] + dpla.at[ix, 'demanda_rango']
                        dpla.at[ix, 'demanda_rango'] = 0

                demanda_faltante = dpla.at[0, 'demanda_rango'] - vol_fresco
                dpla = dpla.sort_values(by=['mes_planificado', 'inicio_rango'])



            print('Algo 3: Venta de ingresos directos')
            #Se traspasan a la tabla de ventas
            tvi_temporal = pd.DataFrame()
            for ix in ingdir.index:
                aux_tvi = pd.DataFrame([[ingdir.at[ix, 'id_ingreso'], mes, ingdir.at[ix, 'volumen'], 0]], columns=['id_ingreso', 'mes_planificado', 'vol_venta', 'vol_almacen'] )
                tvi_temporal = pd.concat([tvi_temporal, aux_tvi], ignore_index=True)
            tabla_ventas_ingresos = pd.concat([tabla_ventas_ingresos, tvi_temporal], ignore_index=True)

            demanda_faltante = 20
            print(demanda_faltante)



            print('Algo 4: Venta de ingresos libres')
            tvi_temporal = pd.DataFrame()
            for ix in inglib.index:
                if demanda_faltante >= inglib.at[ix, 'volumen']:
                    aux_tvi = pd.DataFrame([[inglib.at[ix, 'id_ingreso'], mes, inglib.at[ix, 'volumen'], 0]], columns=['id_ingreso', 'mes_planificado', 'vol_venta', 'vol_almacen'] )
                    tvi_temporal = pd.concat([tvi_temporal, aux_tvi], ignore_index=True)
                    demanda_faltante = demanda_faltante - inglib.at[ix, 'volumen']

                else:
                    aux_tvi = pd.DataFrame([[inglib.at[ix, 'id_ingreso'], mes, demanda_faltante, inglib.at[ix, 'volumen'] - demanda_faltante]], columns=['id_ingreso', 'mes_planificado', 'vol_venta', 'vol_almacen'] )
                    tvi_temporal = pd.concat([tvi_temporal, aux_tvi], ignore_index=True)
                    demanda_faltante = 0
    
            tabla_ventas_ingresos = pd.concat([tabla_ventas_ingresos, tvi_temporal], ignore_index=True)





            print('Algo 5: Se completa demanda faltante en primer rango (0-3) desde almacenes')

            alma = alma.sort_values(by=['prioridad', 'antiguedad'], ascending=[True, False])

            #Solo se itera sobre el primer rango.
            aux_rangos = rangos_demanda[rangos_demanda['id_tipo_rango'] == 1].reset_index(drop=True)
            aux_rangos = np.arange(aux_rangos.at[0, 'fin_rango'], aux_rangos.at[0, 'inicio_rango']-1, -1)
            aux_rangos


            contador_volumen = 0
            for r in aux_rangos:
                if demanda_faltante == 0:
                    break
                for ix in alma.index:
                    if alma.at[ix,'antiguedad'] == str(r):
                        contador_volumen = contador_volumen + alma.at[ix, 'stock_inicial']
                        # print(r, alma_aux.at[ix, 'id_almacen'], contador_volumen)
                        if contador_volumen < demanda_faltante:
                            alma.at[ix,'ventas'] = alma.at[ix, 'stock_inicial']
                        else:
                            diferencia = contador_volumen - demanda_faltante
                            alma.at[ix, 'ventas'] = alma.at[ix, 'stock_inicial'] - diferencia
                            demanda_faltante = 0
                            break

            if demanda_faltante > contador_volumen:
                demanda_faltante = demanda_faltante - contador_volumen

            print(demanda_faltante)
            #Aqui podria ir la produccion
            
            palanca = False
            if palanca == True:
                print('Algo 6: Se completa la demanda faltante para el resto de los rangos (4-6 en adelante)')
                aux_rangos = rangos_demanda[rangos_demanda['id_tipo_rango'] == 1].reset_index(drop=True)
                for ix in aux_rangos.index[1:]:
                    it_rangos = np.arange(aux_rangos.at[ix, 'fin_rango'], aux_rangos.at[ix, 'inicio_rango']-1, -1)
                    contador_volumen = 0

                    for r in it_rangos:
                        if demanda_faltante == 0:
                            break
                        for ix in alma.index:
                            if alma.at[ix,'antiguedad'] == str(r):
                                contador_volumen = contador_volumen + alma.at[ix, 'stock_inicial']
                                print(r, alma.at[ix, 'id_almacen'], contador_volumen)
                                if contador_volumen < demanda_faltante:
                                    alma.at[ix,'ventas'] = alma.at[ix, 'stock_inicial']
                                else:
                                    diferencia = contador_volumen - demanda_faltante
                                    alma.at[ix, 'ventas'] = alma.at[ix, 'stock_inicial'] - diferencia
                                    demanda_faltante = 0
                                    break

                    if demanda_faltante > contador_volumen:
                        demanda_faltante = demanda_faltante - contador_volumen


            